In [1]:
import rolch
import numpy as np
from sklearn.datasets import load_diabetes, make_regression
import matplotlib.pyplot as plt

np.set_printoptions(precision=3, suppress=True)
print(rolch.__version__)

0.1.2


# Batch Estimation

In [2]:
## Diabetes data set
## Add intercept (will not be regularized)

X, y = load_diabetes(return_X_y=True)
X = np.hstack((np.ones((X.shape[0], 1)), X))

## OLS

In [3]:
online_gamlss_ols = rolch.OnlineGamlss(
    distribution=rolch.DistributionNormal(), 
    method="ols",
    rss_tol_inner=np.inf
)

online_gamlss_ols.fit(y, X, X)

print("OLS Coefficients \n")
print(np.vstack(online_gamlss_ols.betas).T)

OLS Coefficients 

[[151.658   3.94 ]
 [  0.636  -0.066]
 [-11.504  -0.114]
 [ 22.429   0.044]
 [ 16.494   0.107]
 [-25.472  -0.416]
 [ 12.469   0.434]
 [ -0.437  -0.071]
 [  6.951  -0.172]
 [ 30.482   0.208]
 [  2.897   0.056]]


## LASSO

In [4]:
dist = rolch.DistributionT()

online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=dist,
    method="lasso",
    estimation_kwargs={
        "ic" : {i: "bic" for i in range(dist.n_params)}, # Change the IC if you like
        "lambda_eps" : {i : 1e-4 for i in range(dist.n_params)} # Change the design of the lambda grid
    },
    rss_tol_inner=np.inf
)
online_gamlss_lasso.fit(y, X, X, X)

print("LASSO Coefficients \n")
print(np.vstack(online_gamlss_lasso.betas).T)

LASSO Coefficients 

[[151.969   3.973  25.   ]
 [ -0.     -0.     -0.   ]
 [ -9.815  -0.     -0.   ]
 [ 24.232   0.044  -0.   ]
 [ 13.786   0.      0.   ]
 [ -4.886  -0.     -0.   ]
 [ -0.      0.      0.   ]
 [-10.781  -0.026   0.   ]
 [  0.      0.      0.   ]
 [ 24.851   0.      0.   ]
 [  2.192   0.      0.   ]]


# Incremental Fit

In [5]:
online_gamlss_lasso = rolch.OnlineGamlss(
    distribution=rolch.DistributionT(), 
    method="lasso", 
    estimation_kwargs={"ic" : {i: "bic" for i in range(dist.n_params)}},
)

online_gamlss_lasso.fit(
    y[:-1], 
    X[:-1, :], 
    X[:-1, :], 
    X[:-1, :]
)

print("Coefficients for the first N-1 observations \n")
print(np.vstack(online_gamlss_lasso.betas).T)

Coefficients for the first N-1 observations 

[[152.023   3.917   2.737]
 [ -0.     -0.      0.   ]
 [-10.774  -0.      0.   ]
 [ 24.566   0.035  -0.277]
 [ 14.204   0.     -0.   ]
 [ -5.537   0.      0.   ]
 [ -0.     -0.      0.   ]
 [-10.838  -0.      0.   ]
 [  0.      0.     -0.   ]
 [ 25.509   0.     -0.   ]
 [  1.748   0.     -0.143]]


In [6]:
online_gamlss_lasso.update(
    y[[-1]], 
    X[[-1], :], 
    X[[-1], :], 
    X[[-1], :]
)

print("\nCoefficients after update call \n")
print(np.vstack(online_gamlss_lasso.betas).T)


Coefficients after update call 

[[152.144   3.933   3.475]
 [ -0.71   -0.045   0.511]
 [-12.521  -0.098   0.618]
 [ 24.582   0.036  -0.223]
 [ 15.341   0.067  -0.232]
 [-33.336  -0.412   2.878]
 [ 19.242   0.388  -2.365]
 [  2.281  -0.002  -0.51 ]
 [  7.311  -0.089   0.267]
 [ 35.07    0.19   -1.478]
 [  2.397   0.044  -0.235]]
